In [1]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import dill as pickle
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import const
from romar import utils
from romar import backend as bkd
from romar.systems import BoxAd
from romar.roms import CoBRAS

import romar.postproc.plotting as pltt

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": 1e-16,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/ROMAr/romar/examples/database"

In [5]:
system = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  rad_dtb=path_to_dtb + "/rates/rad_fit.p",
  use_rad=True,
  use_proj=False,
  use_tables=False
)
system.compute_c_mat(max_mom=2, state_specs=False, include_temp=False)

In [6]:
rho = 1e-3
T = 5.5e4
Te = 3e2
quad_mu = {
  "x": np.array([rho, T, Te]).reshape(1,-1),
  "w": np.ones((1,1))
}
tend = 1e-1

In [ ]:
t = system.get_tgrid(1e-13, tend, 200)
y0, rho = system.equil.get_init_sol(quad_mu["x"].squeeze())
y0

In [ ]:
yfom, _ = system.solve_fom(t, y0, rho)

In [9]:
w_fom = copy.deepcopy(yfom[:system.mix.nb_comp])
n_fom = system.mix.get_n(bkd.to_torch(w_fom)).numpy()
T_fom = copy.deepcopy(yfom[system.mix.nb_comp:])
T_fom[-1] = system.mix.get_Te(pe=T_fom[-1], ne=n_fom[-1])

In [10]:
xref = np.mean(yfom, axis=1)
xscale = np.std(yfom, axis=1)
# xref[-3:] = 0.0
# xscale[-3:] = 1.0

In [11]:
# xref = np.linalg.norm(yfom, ord=2, axis=1)
# xscale = np.std(yfom, axis=1)
xref[0] = 0.0
xref[-3:] = 0.0
xscale[0] = 1.0
xscale[-3:] = 1.0
xref = None
xscale = None

In [12]:
# xscale = 1.0/xref
# xref = None

In [13]:
cobras = CoBRAS(
  system=system,
  tgrid={"start": 1e-12, "stop": 1e0, "num": 51},
  quad_mu=quad_mu,
  xref=xref,
  xscale=xscale,
  path_to_saving="./",
  saving=True
)

In [ ]:
X, Y = cobras.compute_cov_mats(
  nb_meas=10
)
X.shape, Y.shape

In [15]:
# W = np.diag(1.0/xref)
# Ws = np.sqrt(W)
# X = Ws @ X
# Y = Ws @ Y

In [16]:
cobras.compute_modes(
  X=X,
  Y=Y,
  xnot=[0,-3,-2,-1],
  pod=True
)

In [17]:
bases = pickle.load(open("./cobras_bases.p", "rb"))

In [18]:
for i in range(15):
  nb = str(i+1)
  b = bases["phi"][:,i]
  pltt.plot_dist_2d(
    x=np.arange(len(b)),
    y=b,
    labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % ("phi", nb)],
    scales=["linear", "linear"],
    markersize=1,
    # figname=path + f"/{name}_{nb.zfill(2)}",
    save=False,
    show=False
  )

In [19]:
for i in range(15):
  nb = str(i+1)
  b = bases["psi"][:,i]
  pltt.plot_dist_2d(
    x=np.arange(len(b)),
    y=b,
    labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % ("psi", nb)],
    scales=["linear", "linear"],
    markersize=1,
    # figname=path + f"/{name}_{nb.zfill(2)}",
    save=False,
    show=False
  )

In [44]:
rdim = 7

# bases = pickle.load(open("./pod_bases.p", "rb"))
# phi, psi = bases["phi"][:,:rdim], bases["phi"][:,:rdim]

bases = pickle.load(open("./cobras_bases.p", "rb"))
phi, psi = bases["phi"][:,:rdim], bases["psi"][:,:rdim]

system.set_rom(phi, psi, mask="./rom_mask.txt", xref=xref, xscale=xscale)

In [45]:
# I = np.eye(system.nb_eqs)
# system.set_rom(I, I, mask=np.diag(I), xref=xref, xscale=xscale)

In [ ]:
yrom, _ = system.solve_rom(t, y0, rho)
nt = len(yrom.T)
nt

In [47]:
w_rom = copy.deepcopy(yrom[:system.mix.nb_comp])
n_rom = system.mix.get_n(bkd.to_torch(w_rom)).numpy()
T_rom = copy.deepcopy(yrom[system.mix.nb_comp:])
T_rom[-1] = system.mix.get_Te(pe=T_rom[-1], ne=n_rom[-1])

In [ ]:
for i in range(2):
  plt.semilogx(t[1:nt], T_fom[i][1:nt])
  plt.semilogx(t[1:nt], T_rom[i][1:nt], ls="--", color=plt.gca().lines[-1].get_color())
plt.show()
plt.close()

In [49]:
for k in ("Ar", "Arp"):
  s = system.mix.species[k]
  pltt.plot_mom_evolution(
    path=f"./figs/{k}/",
    t=t[1:nt],
    n_m={"FOM": bkd.to_torch(n_fom.T[1:nt,s.indices]), "ROM": bkd.to_torch(n_rom.T[1:nt,s.indices])},
    molecule=s,
    molecule_label="{%s}" % k,
    tlim=None,
    ylim_err=None,
    err_scale="linear",
    hline=None,
    max_mom=2
  )

In [ ]:
for s in system.mix.species.values():
  print(s.name)
  for i in s.indices:
    plt.loglog(t[1:nt], n_fom[i][1:nt])
    plt.loglog(t[1:nt], n_rom[i][1:nt], ls="--", color=plt.gca().lines[-1].get_color())
  plt.show()
  plt.close()